<html>
<center>
</h4>
<font color='blod' size="+2">Task Description!</font></h4>
</center>
<div class="markdown-google-sans">


</div>
<p>In this Notebook, we will fine-tune the models BERT and BioBERT for the NER test on JNLPBA corpora.
<!-- TODO(b/319266067) remove temporary advert after a few weeks. -->
<div class="markdown-google-sans">
  <h2>Biomedical corpora : </h2>
  <ul>
  <li><a href="https://huggingface.co/datasets/ncbi_disease#dataset-card-for-ncbi-disease">NCBI Disease</a></li>
  <p>This dataset contains the disease name and concept annotations of the NCBI disease corpus, a collection of 793 PubMed abstracts fully annotated at the mention and concept level to serve as a research resource for the biomedical natural language processing community.</p>
  <div>
  <center>
  <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
    <th>Test</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>5433</td>
    <td>924</td>
    <td>941</td>
  </tr>
</table>
</div>
   <li><a href="https://huggingface.co/datasets/jnlpba">JNLPBA</a></li>
   The data came from the GENIA version 3.02 corpus (Kim et al., 2003). This was formed from a controlled search on MEDLINE using the MeSH terms human, blood cells and transcription factors. From this search 2,000 abstracts were selected and hand annotated according to a small taxonomy of 48 classes based on a chemical classification. Among the classes, 36 terminal classes were used to annotate the GENIA corpus.
   <div>
   <center>
   <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>37094</td>
    <td>7714</td>
  </tr>
</table>

   </div>
   <li><a href="https://huggingface.co/datasets/linnaeus">LINNAEUS</a></li>
   The LINNAEUS corpus consists of 100 full-text documents from the PMCOA document set which were randomly selected. All mentions of species terms were manually annotated and normalized to the NCBI taxonomy IDs of the intended species.
   <div>
   <center>
   <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
    <th>Test</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>11936</td>
    <td>4079</td>
    <td>7143</td>
  </tr>
</table>

   </div>
  <li><a href="https://huggingface.co/datasets/bc2gm_corpus/viewer/bc2gm_corpus">BC2GM</a></li>
  Created by Smith et al. at 2008, the BioCreative II Gene Mention Recognition (BC2GM) Dataset contains data where participants are asked to identify a gene mention in a sentence by giving its start and end characters. The training set consists of a set of sentences, and for each sentence a set of gene mentions (GENE annotations)
  <div>
  <center>
  <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Val</th>
    <th>Test</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>12500</td>
    <td>2500</td>
    <td>5000</td>
  </tr>
</table>

  </div>
  </ul>

</div>
</html>

# BERT

<font color='green'>Preparing the data</font>
<p>First things, we need a dataset suitable for token classification. In this section we will use the NCBI Disease data.</p>
<p>To load the NCBI Disease  dataset, we use the load_dataset() method from the huggingface 🤗 Datasets library</p>

In [2]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install seqeval
import evaluate
import numpy as np
import torch
from accelerate import Accelerator
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer
from datasets import load_dataset

In [3]:
raw_datasets = load_dataset("jnlpba")

Generating train split:   0%|          | 0/37094 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7714 [00:00<?, ? examples/s]

Dataset jnlpba downloaded and prepared to /root/.cache/huggingface/datasets/jnlpba/jnlpba/1.0.0/3062f220823930cffde7976b694aa67bac3b06c322a02ced92d3761519810ce4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 37094
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 7714
    })
})

**Data Fields**

    id: Sentence identifier.
    tokens: Array of tokens composing a sentence.
    ner_tags: Array of tags, where 0 indicates no disease mentioned, 1 signals the first token of a disease and 2 the subsequent disease tokens.


Let’s have a look at the first element of the training set:

In [5]:
print(raw_datasets["train"][0]["tokens"])

['IL-2', 'gene', 'expression', 'and', 'NF-kappa', 'B', 'activation', 'through', 'CD28', 'requires', 'reactive', 'oxygen', 'production', 'by', '5-lipoxygenase', '.']


Let’s have a look at the first Annotation of the training set:

In [6]:
raw_datasets["train"][0]["ner_tags"]

[1, 2, 0, 0, 9, 10, 0, 0, 9, 0, 0, 0, 0, 0, 9, 0]

In [5]:

ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(num_classes=11, names=['O', 'B-DNA', 'I-DNA', 'B-RNA', 'I-RNA', 'B-cell_line', 'I-cell_line', 'B-cell_type', 'I-cell_type', 'B-protein', 'I-protein'], id=None), length=-1, id=None)

In [6]:

label_names = ner_feature.feature.names
label_names

['O',
 'B-DNA',
 'I-DNA',
 'B-RNA',
 'I-RNA',
 'B-cell_line',
 'I-cell_line',
 'B-cell_type',
 'I-cell_type',
 'B-protein',
 'I-protein']

<font color='green'>Processing the data</font>

To begin, let’s create a tokenizer object.

we will be using a BERT pretrained model, so we’ll start by downloading and caching the associated tokenizer:


In [19]:
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

we will tokenize the first token in our corpus

In [20]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', 'IL', '-', '2', 'gene', 'expression', 'and', 'N', '##F', '-', 'ka', '##ppa', 'B', 'activation', 'through', 'CD', '##28', 'requires', 'reactive', 'oxygen', 'production', 'by', '5', '-', 'lip', '##ox', '##y', '##gen', '##ase', '.', '[SEP]']


As we can see, the tokenizer added the special tokens used by the model ([CLS] at the beginning and [SEP] at the end)

The  Identification, was tokenized into 3 subwords, I,  ##dent and ##ification'. This introduces a mismatch between our inputs and the labels:

In [7]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True,is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [22]:

tokenized_datasets = raw_datasets.map(tokenize_and_align_labels,batched=True)

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [23]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 37094
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7714
    })
})

In [24]:
print(tokenized_datasets['train'][0]['input_ids'])
print(len(tokenized_datasets['train'][0]['input_ids']))

[101, 15393, 118, 123, 5565, 2838, 1105, 151, 2271, 118, 24181, 13059, 139, 14915, 1194, 2891, 24606, 5315, 26844, 7621, 1707, 1118, 126, 118, 4764, 10649, 1183, 4915, 6530, 119, 102]
31


In [25]:

print(tokenized_datasets['train'][0]['labels'])
print(len(tokenized_datasets['train'][0]['labels']))

[-100, 1, -100, -100, 2, 0, 0, 9, -100, -100, -100, -100, 10, 0, 0, 9, -100, 0, 0, 0, 0, 0, 9, -100, -100, -100, -100, -100, -100, 0, -100]
31


In [26]:
example = tokenized_datasets['train'][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
len(tokens),len(tokenized_datasets['train'][0]['labels'])

(31, 31)

In [27]:

tokenized_datasets=tokenized_datasets.remove_columns(['id', 'tokens', 'ner_tags'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 37094
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7714
    })
})

DataCollatorWithPadding  only pads the inputs (input IDs, attention mask, and token type IDs). Here our labels should be padded the exact same way as the inputs so that they stay the same size, using -100 as a value so that the corresponding predictions are ignored in the loss computation.

In [28]:


data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [29]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    1, -100, -100,    2,    0,    0,    9, -100, -100, -100, -100,
           10,    0,    0,    9, -100,    0,    0,    0,    0,    0,    9, -100,
         -100, -100, -100, -100, -100,    0, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100],
        [-100,    0, -100,    0,    0,    9, -100,   10,   10,    0,    0,    0,
            0, -100, -100,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    9, -100, -100, -100, -100, -100,    0,    9, -100, -100,    0,
            0,    0,    0,    0, -100]])

Metrics

To have the Trainer compute a metric every epoch, we will need to

 define a compute_metrics() function that takes the arrays of

 predictions and labels, and returns a dictionary with the metric names and values.

The traditional framework used to evaluate token classification prediction is seqeval. To use this metric, we first need to install the seqeval library:

In [8]:

metric = evaluate.load("seqeval")

In [9]:


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [10]:

labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-DNA',
 'I-DNA',
 'O',
 'O',
 'B-protein',
 'I-protein',
 'O',
 'O',
 'B-protein',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-protein',
 'O']

In [11]:

id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [12]:
id2label

{0: 'O',
 1: 'B-DNA',
 2: 'I-DNA',
 3: 'B-RNA',
 4: 'I-RNA',
 5: 'B-cell_line',
 6: 'I-cell_line',
 7: 'B-cell_type',
 8: 'I-cell_type',
 9: 'B-protein',
 10: 'I-protein'}

In [17]:
label2id

{'O': 0,
 'B-DNA': 1,
 'I-DNA': 2,
 'B-RNA': 3,
 'I-RNA': 4,
 'B-cell_line': 5,
 'I-cell_line': 6,
 'B-cell_type': 7,
 'I-cell_type': 8,
 'B-protein': 9,
 'I-protein': 10}

**Defining the model**

In [36]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
#just check the number of labels
model.config.num_labels

11

In [9]:

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()


#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))939330
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


**Fine-tuning the model**

In [39]:

args = TrainingArguments(
    "disease",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    optim='adamw_torch',
    num_train_epochs=8,
    weight_decay=0.01,
)

In [40]:

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    
)
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.144100,0.193504,0.672283,0.764719,0.715528,0.937945
2,0.094700,0.231472,0.672086,0.782729,0.723200,0.936322
3,0.066100,0.275013,0.668760,0.774532,0.717770,0.936084
4,0.040300,0.320632,0.676900,0.771300,0.721023,0.935827
5,0.026500,0.380830,0.663355,0.763219,0.709792,0.933580
6,0.018200,0.431564,0.671051,0.771531,0.717792,0.935253
7,0.012100,0.458391,0.671437,0.769106,0.716961,0.935193
8,0.008900,0.481910,0.671434,0.769568,0.717160,0.934936


TrainOutput(global_step=18552, training_loss=0.05893013649790798, metrics={'train_runtime': 3126.4488, 'train_samples_per_second': 94.917, 'train_steps_per_second': 5.934, 'total_flos': 1.2818306163393552e+16, 'train_loss': 0.05893013649790798, 'epoch': 8.0})

In [41]:
eval_metrics = trainer.state.log_history

In [42]:
eval_metrics[-2]

{'eval_loss': 0.48191022872924805,
 'eval_precision': 0.6714343271555198,
 'eval_recall': 0.7695682290464096,
 'eval_f1': 0.7171597633136094,
 'eval_accuracy': 0.934936014806164,
 'eval_runtime': 31.4202,
 'eval_samples_per_second': 245.511,
 'eval_steps_per_second': 15.372,
 'epoch': 8.0,
 'step': 18552}

# Biobert

In [13]:
biobert_model_checkpoint = "dmis-lab/biobert-v1.1"
biobert_tokenizer = AutoTokenizer.from_pretrained(biobert_model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = biobert_tokenizer(examples["tokens"], truncation=True,is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [15]:

tokenized_datasets_biobert = raw_datasets.map(tokenize_and_align_labels,batched=True)

  0%|          | 0/38 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

In [16]:
tokenized_datasets_biobert=tokenized_datasets_biobert.remove_columns(['id', 'tokens', 'ner_tags'])
tokenized_datasets_biobert


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 37094
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 7714
    })
})

In [17]:
data_collator = DataCollatorForTokenClassification(tokenizer=biobert_tokenizer)


model = AutoModelForTokenClassification.from_pretrained(
    biobert_model_checkpoint,num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
)

args = TrainingArguments(
    "disease1",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    optim='adamw_torch',
    num_train_epochs=8,
    weight_decay=0.01,
)


pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets_biobert["train"],
    eval_dataset=tokenized_datasets_biobert["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=biobert_tokenizer,
    
)
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.139900,nan,0.691171,0.788040,0.736433,0.941963
2,0.106400,nan,0.691980,0.806858,0.745017,0.938845
3,0.077300,nan,0.686106,0.785038,0.732246,0.938519
4,0.051500,nan,0.701742,0.781459,0.739458,0.939588
5,0.037700,nan,0.679747,0.781921,0.727263,0.936638
6,0.028500,nan,0.687105,0.778804,0.730087,0.937242
7,0.020800,nan,0.685946,0.780420,0.730140,0.937697
8,0.018000,nan,0.682354,0.781690,0.728652,0.937311


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=9280, training_loss=0.06626035714971608, metrics={'train_runtime': 3875.8211, 'train_samples_per_second': 76.565, 'train_steps_per_second': 2.394, 'total_flos': 1.4619514098481968e+16, 'train_loss': 0.06626035714971608, 'epoch': 8.0})